In [5]:
import sys
sys.path.append("ALL_IN_ONE/Pythia8")
sys.path.append("ALL_IN_ONE/LSD")
sys.path.append("ALL_IN_ONE/FIT_FUNC")
import os
from tqdm import tqdm
import ALL_IN_ONE.Pythia8.loop as loop
import ALL_IN_ONE.Pythia8.one_key_run as okr 
import ALL_IN_ONE.Pythia8.run_save as rs
import ALL_IN_ONE.Pythia8.analyse_data as ad
import ALL_IN_ONE.Pythia8.detection as detection
import ALL_IN_ONE.Pythia8.functions_for_calculation as fcal
# from detection import detect_percise_and_combine, detect_r_and_combine
# from analyse_data import see_ctau_2, print_max_min_log10_sin_theta_2

In [2]:
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
from tqdm import tqdm
# import tkinter as tk
# from tkinter import ttk
from threading import Thread
from tqdm import tqdm
import numpy as np
from PIL import Image
# from run_save import run_save_main41_txt, runtxt_to_csv, add_typed_in_data, add_whether_in_the_detector, add_whether_in_the_detector_without_angle
import os
from ALL_IN_ONE.Pythia8.plotting import plot_llp_decay_in_the_detector
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
from datetime import datetime
import os
import subprocess
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import pandas as pd
import random
from tqdm import tqdm
import glob
from mpl_toolkits.mplot3d import Axes3D
# from functions_for_run import mkdir_1
import re
# from functions_for_read import get_llp
# from functions_for_calculation import calculate_decay_position, whether_in_the_detector_by_position, whether_in_the_detector_by_r


In [3]:
hep.style.use(hep.style.ALICE)

In [ ]:
# /Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/auto_data/Ctau_Br/2024-09-17-csv/original_data

In [20]:
def merge_csv_files(directory, output_file):
    # csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    
    data_frames = []
    
    for file in directory:
        file_path = os.path.join(file)
        df = pd.read_csv(file_path)
        data_frames.append(df)
    

    merged_df = pd.concat(data_frames, ignore_index=True)
    
    merged_df.to_csv(output_file, index=False)




In [ ]:
filename = '/Users/shiyuzhe/Documents/University/LLP/Second_Term/test_file/2025-04-01_detected_combined_precise_file.csv'
df = pd.read_csv(filename)
plt.figure(figsize=(15,10))
# print(fcal.calculate_Br(df['m'], df['theta_input']**2)[101:105])
df['significance'] = df['detector_acceptance']*fcal.calculate_Br(df['m'], df['theta_input']**2, 1, 2)*30*1e8
df = df[df['significance'] > 3]
plt.scatter(df['m'], df['significance'], label='CODEX-b', color='blue', s=20, alpha=0.5)
# plt.scatter(df['m'],np.sin(df['theta_input'])**2, label='CODEX-b', color='blue', s=20, alpha=0.5)
plt.ylabel('theta^2', fontsize = 40)
plt.xlabel('mass / GeV', fontsize = 40)
# plt.xlim(1e8, 1e12)
plt.xlim(0,5)
plt.yscale('log')
plt.tick_params(axis='both', which='major', labelsize=30)
plt.title('CODEX-b Detecor Reach for Higgs Portal', size = 30)
plt.legend()
plt.show()
plt.savefig('2025-03-123_precise_cs_mass.jpg')

# plt.show()
plt.close()

/var/folders/kt/7k22p1ns6l7ggtc7ck9nkqbw0000gn/T/ipykernel_67014/1542538594.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
